# Output PDBs with the *average* escape scores for a given antibody class or plasma/serum group as B factors
This Python Jupyter notebook outputs PDBs with the escape scores as B factors.

Though we will want more elaborate series of commands to codify our visualization of these RBD structures colored by escape, the series of commands below, when executed in a `PyMol` session with one of these PDBs open, will color the RBD surface according to escape scores.

We want to take 

Alternatively, it might be worth it to take a weighted average of the escape fractions for each condition within in a given group. This might be important because some (noisy) sera have relatively high escape fractions across the entire RBD. We currently show these as flat escape profiles in the logo plots, with ylims that are calculated according to specs in the `escape_profiles_config.yaml` file. These ylims are saved to an output CSV file in the `results/escape_profiles/` subdirectory, so it would be possible to normalize to that y-max value before averaging. This may not be necessary, so we should see how it looks first. 

Also, we are only interested in the toal escape at a site, not the max mutation effect. 
     
We write PDBs with B factors indicating the group-average total site escape.

First, import Python modules:

In [1]:
import collections
import copy
import os
import warnings

import Bio.PDB

import dms_variants.pdb_utils

from IPython.display import display, HTML

import pandas as pd

import yaml

Read the configuration file:

In [2]:
with open('../config.yaml') as f:
    config = yaml.safe_load(f)

Read escape profiles config file and configuration for outputting PDBs:

In [3]:
profiles_config = os.path.join('../', (config['escape_profiles_config']))

print(f"Reading escape profiles configuration from {profiles_config}")
with open(profiles_config) as f:
    escape_profiles_config = yaml.safe_load(f)

Reading escape profiles configuration from ../data/escape_profiles_config.yaml


Make output directory:

In [4]:
resultsdir = '../results/moderna_custom_plots/pdb_outputs/'
os.makedirs(resultsdir, exist_ok=True)

Read escape fractions and compute **total** and **maximum** escape at each site, and also the total and maximum escape at each site normalized to be between 0 and 1 for each selection:

In [5]:
print(f"Reading escape fractions from {os.path.join('../', config['escape_fracs'])}")

escape_fracs = (
    pd.read_csv(os.path.join('../', config['escape_fracs']))
    .query('library == "average"')
    .assign(site=lambda x: x['label_site'])
    .groupby(['selection', 'site'])
    .aggregate(total_escape=pd.NamedAgg(config['mut_metric'], 'sum'),
#                max_escape=pd.NamedAgg(config['mut_metric'], 'max')
               )
    .reset_index()
    )

display(HTML(escape_fracs.head().to_html(index=False)))

Reading escape fractions from ../results/escape_scores/escape_fracs.csv


selection,site,total_escape
12C_d152_80,331,0.057853
12C_d152_80,332,0.089158
12C_d152_80,333,0.064947
12C_d152_80,334,0.143838
12C_d152_80,335,0.092717


Now map the escape metrics to the B-factors.
For sites where no mutations have escape scores:
 - In the RBD chain(s) fill the B-factor for non-normalized scores to -1 to enable collapsing to zero or callout as a a separate class, depending how we choose to color sites for different visualizations. 
 - In other chains, always fill missing B factors to 0.  

In [6]:
pdbfile = '../data/pdbs/6M0J.pdb'
assert os.path.isfile(pdbfile)
rbd_chain = config['escape_frac_protein_chain']
assert isinstance(rbd_chain, str)

for name in escape_profiles_config:
    print(f"\nMaking PDB mappings for {name} to {pdbfile}")
    
    # get conditions from escape_profiles_config.yaml
    conditions = escape_profiles_config[name]['conditions'].keys()
    print(f"Making mappings for {len(conditions)} conditions.")
    
    # get escape fracs just for conditions of interest
    df = escape_fracs.query('selection in @conditions')
    
    # assign average total_escape at each site across all the conditions in ab_class
    df = (df
          .groupby(['site'])
          .aggregate(mean_total_escape=pd.NamedAgg('total_escape', 'mean'),
                      )
          .reset_index()
          .drop_duplicates()
         )
    
    # get chains
    print(f'Mapping to the following chain: {rbd_chain}')
    df = df.assign(chain=rbd_chain)
    
    
    # make mappings for each condition and metric
    print(f"  Writing B-factor re-assigned PDBs for {name} to:")

    for metric in ['mean_total_escape']: # keeping this as list because we might need to normalize

        # what do we assign to missing sites?
        missing_metric = collections.defaultdict(lambda: 0)  # non-RBD chains always fill to zero
        missing_metric[rbd_chain] = -1  # missing sites in RBD are -1 for non-normalized metric PDBs

        fname = os.path.join(resultsdir, f"{name}_6m0j_{metric}.pdb")
        print(f"    {fname}")

        dms_variants.pdb_utils.reassign_b_factor(input_pdbfile=pdbfile,
                                                 output_pdbfile=fname,
                                                 df=df,
                                                 metric_col=metric,
                                                 missing_metric=missing_metric)


Making PDB mappings for AZ_cocktail to ../data/pdbs/6M0J.pdb
Making mappings for 3 conditions.
Mapping to the following chain: E
  Writing B-factor re-assigned PDBs for AZ_cocktail to:
    ../results/moderna_custom_plots/pdb_outputs/AZ_cocktail_6m0j_mean_total_escape.pdb

Making PDB mappings for human_sera to ../data/pdbs/6M0J.pdb
Making mappings for 23 conditions.
Mapping to the following chain: E
  Writing B-factor re-assigned PDBs for human_sera to:
    ../results/moderna_custom_plots/pdb_outputs/human_sera_6m0j_mean_total_escape.pdb

Making PDB mappings for NHP_sera to ../data/pdbs/6M0J.pdb
Making mappings for 1 conditions.
Mapping to the following chain: E
  Writing B-factor re-assigned PDBs for NHP_sera to:
    ../results/moderna_custom_plots/pdb_outputs/NHP_sera_6m0j_mean_total_escape.pdb

Making PDB mappings for EZ_7A to ../data/pdbs/6M0J.pdb
Making mappings for 1 conditions.
Mapping to the following chain: E
  Writing B-factor re-assigned PDBs for EZ_7A to:
    ../results/mod